# 04 - Signal Detection
## Semi-Automated Signal Detection for Annotation

This notebook covers:
- Detecting Bluetooth signals automatically
- Generating preliminary bounding boxes
- Creating YOLO format annotations
- Manual verification workflow

In [ ]:
# Setup
import sys
sys.path.append('..')

import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import cv2
from scipy.signal import find_peaks

from src.visualization import plot_spectrogram_with_detections

print("✅ All imports successful!")

## Configuration

In [ ]:
# Paths
SPECTROGRAM_DIR = Path('../data/spectrograms')
ANNOTATION_DIR = Path('../data/annotations')
ANNOTATION_DIR.mkdir(parents=True, exist_ok=True)

# Signal classes
SIGNAL_CLASSES = {
    0: 'bluetooth',
    1: 'wifi',
    2: 'zigbee',
    3: 'drone'
}

# Detection parameters
INTENSITY_THRESHOLD = 180
MAX_WIDTH = 15  # pixels
MIN_HEIGHT = 30  # pixels

print(f"Classes: {list(SIGNAL_CLASSES.values())}")

## Automatic Signal Detection Function

In [ ]:
def detect_narrow_vertical_signals(spectrogram, intensity_threshold=180, 
                                   max_width=15, min_height=30):
    """
    Automatically detect Bluetooth-like signals for annotation assistance
    
    Returns:
        List of bounding boxes in YOLO format
    """
    gray = cv2.cvtColor(spectrogram, cv2.COLOR_RGB2GRAY)
    
    # Threshold high-intensity regions
    _, binary = cv2.threshold(gray, intensity_threshold, 255, cv2.THRESH_BINARY)
    
    # Find connected components
    num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(binary)
    
    bboxes = []
    height, width = spectrogram.shape[:2]
    
    for i in range(1, num_labels):  # Skip background (0)
        x, y, w, h, area = stats[i]
        
        # Filter for Bluetooth characteristics (narrow vertical)
        if w <= max_width and h >= min_height:
            # Convert to YOLO format (normalized)
            x_center = (x + w/2) / width
            y_center = (y + h/2) / height
            w_norm = w / width
            h_norm = h / height
            
            bboxes.append({
                'class_id': 0,  # Bluetooth
                'bbox': [x_center, y_center, w_norm, h_norm],
                'confidence': area / (w * h)  # Density score
            })
    
    return bboxes

def save_yolo_annotation(image_path, bboxes, output_dir):
    """Save YOLO format annotation file"""
    image_name = Path(image_path).stem
    annotation_file = Path(output_dir) / f"{image_name}.txt"
    
    with open(annotation_file, 'w') as f:
        for bbox in bboxes:
            class_id = bbox['class_id']
            x, y, w, h = bbox['bbox']
            f.write(f"{class_id} {x:.6f} {y:.6f} {w:.6f} {h:.6f}\n")

## Test Detection on Sample Image

In [ ]:
# Load sample spectrogram
spec_files = sorted(SPECTROGRAM_DIR.glob('*.png'))

if len(spec_files) > 0:
    sample_file = spec_files[0]
    img = cv2.imread(str(sample_file))
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    # Detect signals
    bboxes = detect_narrow_vertical_signals(
        img_rgb,
        intensity_threshold=INTENSITY_THRESHOLD,
        max_width=MAX_WIDTH,
        min_height=MIN_HEIGHT
    )
    
    print(f"Detected {len(bboxes)} potential Bluetooth signals")
    
    # Visualize
    plot_spectrogram_with_detections(
        img_rgb,
        bboxes,
        title=f"Detections: {sample_file.name}"
    )
else:
    print("⚠️ No spectrograms found")

## Batch Process All Spectrograms

In [ ]:
from tqdm import tqdm

spec_files = sorted(SPECTROGRAM_DIR.glob('*.png'))
total_detections = 0

for spec_file in tqdm(spec_files, desc="Processing spectrograms"):
    img = cv2.imread(str(spec_file))
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    # Detect signals
    bboxes = detect_narrow_vertical_signals(
        img_rgb,
        intensity_threshold=INTENSITY_THRESHOLD,
        max_width=MAX_WIDTH,
        min_height=MIN_HEIGHT
    )
    
    # Save annotation
    if len(bboxes) > 0:
        save_yolo_annotation(spec_file, bboxes, ANNOTATION_DIR)
        total_detections += len(bboxes)

print(f"\n✅ Processed {len(spec_files)} spectrograms")
print(f"Total detections: {total_detections}")
print(f"Average detections per image: {total_detections/len(spec_files):.2f}")
print(f"Annotations saved to {ANNOTATION_DIR}")

## Annotation Statistics

In [ ]:
# Count annotations
annotation_files = sorted(ANNOTATION_DIR.glob('*.txt'))
detections_per_file = []

for annot_file in annotation_files:
    with open(annot_file, 'r') as f:
        num_detections = len(f.readlines())
        detections_per_file.append(num_detections)

if len(detections_per_file) > 0:
    plt.figure(figsize=(10, 6))
    plt.hist(detections_per_file, bins=20, edgecolor='black')
    plt.xlabel('Number of Detections per Image')
    plt.ylabel('Count')
    plt.title('Distribution of Detections Across Spectrograms')
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

## Next Steps

1. Manually verify and correct annotations using LabelImg or Roboflow
2. Continue to notebook 05 for YOLO dataset preparation